In [ ]:
#AUTORA: RAFAELA BERNARDES RABELO
# CÓDIGO PARA PROJEÇÃO DE AÇO TRIMESTRAL - SÉRIES TEMPORAIS E APLICAÇÃO DE REDES NEURAIS COM FATORES EXÓGENOS E CORRELAÇÃO CRUZADA
# SEMPRE INSTALAR: pip install pandas numpy matplotlib scikit-learn tensorflow statsmodels

#-------------------------------------------------------------------------------------------------------------
#------------------------------------------  B I B L I O T E C A  --------------------------------------------
#-------------------------------------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime
#Correlação cruzada
from sklearn.preprocessing import MinMaxScaler
#Casualidade
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
#Modelagem conjunta
from statsmodels.tsa.api import VAR
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#Rede neural
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization, Dropout
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import tempfile
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout


# Definir semente para NumPy para reprodutibilidade
np.random.seed(50)
# Definir semente para TensorFlow para reprodutibilidade
tf.random.set_seed(50)
#-------------------------------------------------------------------------------------------------------------
#------------------------------------------------  D A D O S  ------------------------------------------------
#-------------------------------------------------------------------------------------------------------------
# Valores reais do preço do aço fornecidos
vergalhao_prices = [3266.46,3381.69,3567.99,3415.5,3249.21,3390.66,3441.72,3575.58,3445.86,3246.45,3166.41,3146.4,3047.04,2997.36,2915.25,2913.18,2924.91,2955.27,2791.74,3007.02,2558.52,2386.02,2530.92,2575.77,2486.07,2638.56,2677.89,2615.79,2576.46,2381.19,2238.36,2183.85,2332.89,2340.48,2380.5,2274.24,2406.03,2307.36,2239.05,2096.22,2289.42,2288.73,2058.27,1939.59,2106.57,2001,1726.38,1831.26,1824.36,1798.83,1509.72,1523.52,1578.03,1522.14,1417.95,1295.13,1409.67,1335.15,1239.24,1217.16,1164.03,1270.29,1216.47,1263.39,1569.75,1881.63,1332.39,1523.52,1616.67,1640.82,1495.92,1786.41,2087.94,1980.3,2198.34,2290.8,2092.77,2020.32,1933.38,2087.94,2349.45,2579.91,2386.71,2407.41,2546.1,2496.42,2532.99,2612.34,2237.67,2357.73,2347.38,2383.95,2633.04,2649.6,2560.59,2574.39,2255.61,2161.77,2289.42,2320.47,2249.4,2435.01,2322.54,2472.27,2427.42,2201.79,2274.24,2226.63,2362.56,2455.02,2252.85,2270.1,2177.64,2212.83,2324.61,2366.01,2489.52,2595.78,2451.57,2377.74,2525.4,2881.44,2830.38,3065.67,3258.18,3659.76,3394.11,3478.29,3972.33,3680.46,3897.81,3236.1,3054.63,3138.12,3065.67,3002.88,3412.05,3285.09,3160.2,2980.8,2688.93,2539.2,2637.87,2287.35,2535.75,2807.61,2848.32,2844.87,2862.81,2541.27,2379.12,2584.74,2655.81,2586.12,2553,2573.01,2681.34,2760,2687.55,2635.11]
minerio_ferro_prices = [794.48,833.09,889.02,926.64,861.3,886.55,834.57,811.8,856.85,878.13,878.63,744.48,670.73,674.69,694.98,694.98,715.77,730.62,674.69,666.27,633.11,533.61,491.04,564.05,595.73,636.32,746.76,765.47,692.36,680.33,617.07,568.01,630.78,678.4,664.24,656.22,674.78,672.16,607.02,600.78,553.56,567.17,499.41,459.06,475.45,459.56,407.24,396.4,361.99,341.15,332.64,309.92,281.85,254.23,298.93,308.53,254.97,273.64,278.93,261.86,228.2,195.92,203.79,229.33,274.82,294.92,269.73,254.23,280.22,299.13,280.32,310.71,357.64,394.76,397.98,435.95,429.86,346.35,300.51,277.25,327.79,371.79,345.36,298.53,313.63,352.84,375.51,383.43,345.02,323.09,326.45,320.76,318.53,332.44,340.21,360.41,357.79,342.54,376.35,432.28,424.22,461.54,488.86,540.44,594.1,450,459.9,443.27,419.17,453.07,457.58,427.98,435.7,415.01,455.65,509.6,539.06,606.52,613.7,594.94,616.87,771.41,832.24,819.77,826.16,889.17,1018.36,1062.02,1049.35,788.29,592.27,600.09,470.1,556.88,649.19,702.85,746.66,746.31,660.87,643.5,530.74,518.56,486.63,457.53,461.59,550.84,610.68,622.46,628.95,574.89,520.1,557.22,556.68,541.53,597.91,588.6,645.78,675.03,668.89,630.58]
#Criação do data frame
#Um DataFrame é criado para facilitar a manipulação e análise dos dados.
df = pd.DataFrame({'Minerio_Ferro': minerio_ferro_prices, 'Vergalhao_Aco': vergalhao_prices})

#-------------------------------------------------------------------------------------------------------------
#----------------------------------------  N O R M A L I Z A Ç Ã O  ------------------------------------------
#-------------------------------------------------------------------------------------------------------------  
#Normalização dos dados
#Os dados são normalizados usando MinMaxScaler para colocá-los na mesma escala, facilitando a comparação.
scaler = MinMaxScaler()
df[['Minerio_Ferro_Normalizado', 'Vergalhao_Aco_Normalizada']] = scaler.fit_transform(df[['Minerio_Ferro', 'Vergalhao_Aco']])


#-------------------------------------------------------------------------------------------------------------
#-------------------------------------  C O R R E L A Ç Ã O   C R U Z A D A  ---------------------------------
#-------------------------------------------------------------------------------------------------------------
#A função np.correlate é usada para calcular a correlação cruzada entre as séries temporais normalizadas do minério de ferro e da bobina de aço. 
#O parâmetro mode='same' garante que o resultado tenha o mesmo comprimento que as séries temporais originais.
correlation_matrix = np.corrcoef(df['Minerio_Ferro_Normalizado'], df['Vergalhao_Aco_Normalizada'])
correlation_minerio_aco = correlation_matrix[0, 1]

#-------------------------------------------------------------------------------------------------------------
#------------------- V I S U A L I Z A Ç Ã O   D A    C O R R E L A Ç Ã O   C R U Z A D A  -------------------
#-------------------------------------------------------------------------------------------------------------
print('Correlação cruzada:')
# Análise da correlação
correlation_analysis = ""
# Definindo os limiares para interpretação
strong_threshold = 0.7
moderate_threshold = 0.3

if correlation_minerio_aco >= strong_threshold:
    correlation_analysis = "Correlação muito forte"
elif strong_threshold > correlation_minerio_aco >= moderate_threshold:
    correlation_analysis = "Correlação moderada"
else:
    correlation_analysis = "Correlação fraca"

if correlation_minerio_aco >= 0:
    correlation_analysis += " e positiva."
else:
    correlation_analysis += " e negativa."

print(f"A correlação entre o preço do Minério de Ferro e do Vergalhão é {correlation_minerio_aco:.2f}. {correlation_analysis}")

# Gráfico de dispersão entre as séries temporais normalizadas
plt.figure(figsize=(12, 6))
plt.scatter(df['Minerio_Ferro_Normalizado'], df['Vergalhao_Aco_Normalizada'], label='Dispersão Minério de Ferro-Vergalhão de Aço', marker='o')
plt.xlabel('Minério de Ferro Normalizado')
plt.ylabel('Vergalhão de Aço Normalizada')
plt.title('Gráfico de Dispersão - Minério de Ferro vs. Vergalhão de Aço')
plt.legend()
plt.show()

#-------------------------------------------------------------------------------------------------------------
#----------------------------------------------  C A S U A L I D A D E  --------------------------------------
#-------------------------------------------------------------------------------------------------------------
print(f'Casualidade:')
# Escolha do número máximo de lags
max_lags = 2
granger_test_result = grangercausalitytests(df[['Minerio_Ferro', 'Vergalhao_Aco']], max_lags, verbose=False)

for lag in range(1, max_lags + 1):
    p_value = granger_test_result[lag][0]['ssr_ftest'][1]
    significance = 'Significativo' if p_value < 0.05 else 'Não significativo'
    print(f'Causalidade de Minério de Ferro para Vergalhão de Aço em lag {lag}: P-valor = {p_value} ({significance})')

print('\n')

#-------------------------------------------------------------------------------------------------------------
#----------------------------------  D I V I S Ã O   T R E I N A M E N T O / T E S T E  ----------------------
#-------------------------------------------------------------------------------------------------------------
# Divisão Treinamento/Teste
print('Divisão de treinamento/teste')

# Especificar a proporção de dados de treinamento
train_size = int(len(df) * 0.8)  # 80% para treinamento, 20% para teste
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Remover a coluna 'Mes' do conjunto de treinamento
X_train, y_train = train[['Minerio_Ferro_Normalizado', 'Vergalhao_Aco_Normalizada']], train['Vergalhao_Aco_Normalizada']
# Dividir os dados normalizados em treinamento e teste
X_test, y_test = test[['Minerio_Ferro_Normalizado', 'Vergalhao_Aco_Normalizada']], test['Vergalhao_Aco_Normalizada']

print('Divisão realizada')
print('\n')

# Plotar as séries temporais completas
plt.figure(figsize=(10, 6))

# Série temporal completa para Minerio_Ferro
plt.plot(df.index, df['Minerio_Ferro'], label='Minerio_Ferro', color='blue')
# Série temporal completa para Vergalhao_Aco
plt.plot(df.index, df['Vergalhao_Aco'], label='Vergalhao_Aco', color='orange')

# Linha vertical para indicar a divisão entre treinamento e teste
plt.axvline(x=train.index[-1], color='red', linestyle='--', label='Divisão Treinamento/Teste')
# Preencher a área abaixo da linha vertical (conjunto de teste)
plt.fill_between(df.index, df['Minerio_Ferro'], df['Vergalhao_Aco'], where=(df.index >= train.index[-1]), color='green', alpha=0.3, label='Teste')
# Preencher a área acima da linha vertical (conjunto de treinamento)
plt.fill_between(df.index, df['Minerio_Ferro'], df['Vergalhao_Aco'], where=(df.index < train.index[-1]), color='yellow', alpha=0.3, label='Treinamento')

# Adicionar rótulos e título
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('Divisão Treinamento/Teste')
plt.legend()
plt.show()


#-------------------------------------------------------------------------------------------------------------
#--------- A D A P T A Ç Ã O   D O   C Ó D I G O   P A R A   A R Q U I T E T U R A   E X Ó G E N A -----------
#-------------------------------------------------------------------------------------------------------------
# Adicionar essa linha para preparar X_test_lstm_exog
X_test_lstm_exog = np.column_stack((X_test.values, test['Minerio_Ferro_Normalizado'].values))

# Adicionar essa linha para preparar X_train_lstm_exog
X_train_lstm_exog = np.column_stack((X_train.values, train['Minerio_Ferro_Normalizado'].values))

# Adicionar essa linha para garantir que X_train_lstm_exog seja bidimensional
X_train_lstm_exog = X_train_lstm_exog.reshape((X_train_lstm_exog.shape[0], X_train_lstm_exog.shape[1], 1))

#-------------------------------------------------------------------------------------------------------------
#--------------------------------------------  R E D E  N E U R A L  -----------------------------------------
#-------------------------------------------------------------------------------------------------------------
# Rede Neural com Entrada Exógena
model_lstm_exog = Sequential()

# Ajuste 2: Adicionar BatchNormalization antes da camada LSTM
model_lstm_exog.add(BatchNormalization(input_shape=(X_train_lstm_exog.shape[1], X_train_lstm_exog.shape[2])))


# Ajuste 3: Aumentar o número de unidades LSTM
model_lstm_exog.add(LSTM(units=100, activation='relu', dropout=0.1, recurrent_dropout=0.1))

# Ajuste 4: Adicionar BatchNormalization após a camada LSTM
model_lstm_exog.add(BatchNormalization())

# Ajuste 5: Adicionar Dropout após a camada BatchNormalization
model_lstm_exog.add(Dropout(0.1))

# Ajuste 6: Adicionar uma camada densa adicional
model_lstm_exog.add(Dense(units=100, activation='relu'))

# Ajuste 7: Adicionar BatchNormalization após a camada densa
model_lstm_exog.add(BatchNormalization())

# Ajuste 8: Adicionar Dropout após a camada BatchNormalization
model_lstm_exog.add(Dropout(0.1))

# Ajuste 9: Adicionar a camada de saída
model_lstm_exog.add(Dense(units=1))

# Compile o modelo
model_lstm_exog.compile(optimizer='adam', loss='mean_squared_error')

# Treinamento do modelo
model_lstm_exog.fit(X_train_lstm_exog, y_train.values, epochs=500, batch_size=32, verbose=1)



# Reformular dados de teste para o modelo LSTM
X_test_lstm_exog = X_test_lstm_exog.reshape((X_test_lstm_exog.shape[0], X_test_lstm_exog.shape[1], 1))
y_test_lstm = y_test.values.reshape(-1, 1, 1)

# Prevendo os resultados
forecast_lstm_exog = model_lstm_exog.predict(X_test_lstm_exog).flatten()


# Avaliação do modelo LSTM
mae_lstm_exog = mean_absolute_error(y_test, forecast_lstm_exog)
mse_lstm_exog = mean_squared_error(y_test, forecast_lstm_exog)
rmse_lstm_exog = np.sqrt(mse_lstm_exog)

print(f'MAE LSTM com Entrada Exógena: {mae_lstm_exog:.2f}')
print(f'MSE LSTM com Entrada Exógena: {mse_lstm_exog:.2f}')
print(f'RMSE LSTM com Entrada Exógena: {rmse_lstm_exog:.2f}')

# Plotar resultados
plt.figure(figsize=(12, 6)) 
plt.plot(y_test.index, y_test, label='Real Vergalhao_Aco', color='green')
plt.plot(y_test.index, forecast_lstm_exog, label='Previsto LSTM com Entrada Exógena', color='blue')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('Comparação entre Real e Previsto - LSTM com Entrada Exógena')
plt.legend()
plt.show()



#-------------------------------------------------------------------------------------------------------------
#--------------------------------------------  P R O J E Ç Ã O  ----------------------------------------------
#-------------------------------------------------------------------------------------------------------------
# Rede Neural com Entrada Exógena
model_lstm_exog_fut = Sequential()

# Ajuste 2: Adicionar BatchNormalization antes da camada LSTM
model_lstm_exog_fut.add(BatchNormalization(input_shape=(X_train_lstm_exog.shape[1], X_train_lstm_exog.shape[2])))


# Ajuste 3: Aumentar o número de unidades LSTM
model_lstm_exog_fut.add(LSTM(units=100, activation='relu', dropout=0.1, recurrent_dropout=0.1))

# Ajuste 4: Adicionar BatchNormalization após a camada LSTM
model_lstm_exog_fut.add(BatchNormalization())

# Ajuste 5: Adicionar Dropout após a camada BatchNormalization
model_lstm_exog_fut.add(Dropout(0.1))

# Ajuste 6: Adicionar uma camada densa adicional
model_lstm_exog_fut.add(Dense(units=100, activation='relu'))

# Ajuste 7: Adicionar BatchNormalization após a camada densa
model_lstm_exog_fut.add(BatchNormalization())

# Ajuste 8: Adicionar Dropout após a camada BatchNormalization
model_lstm_exog_fut.add(Dropout(0.1))

# Ajuste 9: Adicionar a camada de saída
model_lstm_exog_fut.add(Dense(units=1))

# Compile o modelo
model_lstm_exog_fut.compile(optimizer='adam', loss='mean_squared_error')

# Treinamento do modelo apenas uma vez
model_lstm_exog_fut.fit(X_train_lstm_exog, y_train.values, epochs=500, batch_size=32, verbose=1, validation_data=(X_test_lstm_exog, y_test.values))

# Número de períodos para projetar
num_periodos = 3

# Inicialize uma lista para armazenar as projeções
projecoes = []

# Use os dados de teste para iniciar as projeções
x_projecao = X_test_lstm_exog.copy()

for _ in range(num_periodos):
    # Remodele a última observação no conjunto de treinamento para o modelo
    x_projecao_atual = x_projecao[-1].reshape((1, x_projecao.shape[1], 1))

    # Faça a previsão para o próximo período
    previsao_proximo_periodo = model_lstm_exog_fut.predict(x_projecao_atual).flatten()

    # Adicione a previsão à lista
    projecoes.append(previsao_proximo_periodo[0])

    # Atualize x_projecao com a nova observação
    x_projecao_atual[0, :, 0] = previsao_proximo_periodo[0]  # Atualize a última entrada com a nova projeção
    x_projecao = np.concatenate([x_projecao, x_projecao_atual], axis=0)

# Exiba as projeções
print('Projeções:')
print(projecoes)

# Salve os valores originais do preço do Vergalhao_Aco antes da normalização
min_max_values = df[['Vergalhao_Aco']].agg(['min', 'max']).values.flatten()

# Desnormalize todas as projeções
projecoes_desnormalizadas = np.array(projecoes) * (min_max_values[1] - min_max_values[0]) + min_max_values[0]

# Exiba as projeções desnormalizadas
print('Projeções Desnormalizadas:')
print(projecoes_desnormalizadas)




#-------------------------------------------------------------------------------------------------------------
#--------------------------------------------  P L O T A G E M  ----------------------------------------------
#-------------------------------------------------------------------------------------------------------------
# Plotar as séries temporais completas
plt.figure(figsize=(12, 6))

# Histórico
plt.plot(df.index, df['Vergalhao_Aco'], label='Histórico Vergalhao_Aco', color='blue')

# Linha vertical para indicar a divisão entre treinamento e teste
plt.axvline(x=train.index[-1], color='red', linestyle='--', label='Divisão Treinamento/Teste')

# Preencher a área abaixo da linha vertical (conjunto de teste)
plt.fill_between(df.index, df['Vergalhao_Aco'], where=(df.index >= train.index[-1]), color='green', alpha=0.3, label='Teste')

# Preencher a área acima da linha vertical (conjunto de treinamento)
plt.fill_between(df.index, df['Vergalhao_Aco'], where=(df.index < train.index[-1]), color='yellow', alpha=0.3, label='Treinamento')

# Projeção
# Adicionar projeções ao gráfico, começando do último ponto do histórico até o último ponto do histórico mais o número de períodos de projeção
projections_index = np.arange(len(df) - 1, len(df) - 1 + num_periodos + 1)
projections_values = [df['Vergalhao_Aco'].iloc[-1]] + projecoes_desnormalizadas.tolist()
plt.plot(projections_index, projections_values, color='red', linestyle='--', label='Projeção')

# Preencher a área abaixo da linha de projeção com a cor lilás
plt.fill_between(projections_index, projections_values, color='purple', alpha=0.3, label='Projeção')

# Adicionar rótulos e título
plt.xlabel('Período')
plt.ylabel('Valor')
plt.title('Histórico e Projeção do Vergalhao_Aco')
plt.legend()
plt.show()


# Calcular erros para projeções futuras
for i in range(num_periodos):
    # Índices relativos às projeções
    idx_projecao = np.arange(len(df) - num_periodos + i, len(df))

    erro_mae = mean_absolute_error(df['Vergalhao_Aco'].iloc[idx_projecao], projecoes_desnormalizadas[i:])
    erro_mse = mean_squared_error(df['Vergalhao_Aco'].iloc[idx_projecao], projecoes_desnormalizadas[i:])
    erro_rmse = np.sqrt(erro_mse)
    
    print(f'Erro MAE para o período {i + 1}: {erro_mae:.2f}')
    print(f'Erro MSE para o período {i + 1}: {erro_mse:.2f}')
    print(f'Erro RMSE para o período {i + 1}: {erro_rmse:.2f}')



# Plotar as séries temporais completas
plt.figure(figsize=(12, 6))

# Preencher a área abaixo da linha vertical (conjunto de teste)
plt.fill_between(df.index, df['Vergalhao_Aco'], where=(df.index >= train.index[-1]), color='green', alpha=0.3, label='Teste')

# Projeção
# Adicionar projeções ao gráfico, começando do último ponto do histórico até o último ponto do histórico mais o número de períodos de projeção
projections_index = np.arange(len(df) - 1, len(df) - 1 + num_periodos + 1)
projections_values = [df['Vergalhao_Aco'].iloc[-1]] + projecoes_desnormalizadas.tolist()
plt.plot(projections_index, projections_values, color='red', linestyle='--', label='Projeção')

# Preencher a área abaixo da linha de projeção com a cor lilás
plt.fill_between(projections_index, projections_values, color='purple', alpha=0.3, label='Projeção')

# Adicionar rótulos e título
plt.xlabel('Período')
plt.ylabel('Valor')
plt.title('Histórico e Projeção do Vergalhao_Aco')
plt.legend()
plt.show()
